In [1]:
import numpy as np
import h5py
import pandas as pd
#import tensorflow as tf
#from tensorflow import keras
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt 
import seaborn as sns 

plt.rcParams["figure.figsize"] = [16,9]
sns.set_style('whitegrid')

ModuleNotFoundError: No module named 'sklearn'

In [5]:
#what is our input layer sie? this is = to the length of the wl_grid
#load wl_grid
hf_spec = h5py.File("/home/pravn/Dropbox/Masters/Research Data/resampled_emission_spectra.h5", "r")
hf_grid = h5py.File("/home/pravn/Dropbox/Masters/Research Data/wl_grid.h5", "r")

wl_grid = np.array(hf_grid["wl_grid"])
df = pd.DataFrame(hf_spec["spectra"])

In [6]:
len(wl_grid)

4459

In [7]:
df_labeled = pd.read_csv("/home/pravn/Dropbox/Masters/galah-test-nut/best_emission_candidates_spectra_labels.csv")

In [8]:
df_labeled["label"]

0       1
1       2
2       4
3       2
4       2
       ..
6972    3
6973    1
6974    1
6975    1
6976    2
Name: label, Length: 6977, dtype: int64

In [9]:
df["label"] = df_labeled["label"]

In [10]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,4450,4451,4452,4453,4454,4455,4456,4457,4458,label
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2


In [2]:
import os, sys
from keras.layers import Input, Dense, Dropout
from keras.models import Model, Sequential, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.advanced_activations import PReLU
from keras import backend as K
#from keras.utils import plot_model
from keras.utils.vis_utils import plot_model

2022-04-11 18:47:57.441161: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-11 18:47:57.441231: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
activation = None  # PReLU if set to None
dropout_rate = 0  # from 0 to 1
n_l_e = 5
decoded_layer_name = 'encoded'
n_wvl = 4459

# compute number of nodes in every connected layer
n_l_1 = int(n_wvl * 0.75)
n_l_2 = int(n_wvl * 0.50)
n_l_3 = 0  # int(n_wvl * 0.40)
n_l_4 = 0  # int(n_wvl * 0.20)
n_l_5 = int(n_wvl * 0.25)
n_l_6 = int(n_wvl * 0.10)

In [5]:
autoencoder = Sequential()

if n_l_1 > 0:
    autoencoder.add(Dense(n_l_1, input_shape=(n_wvl,), activation=activation, name='E_1'))
    if dropout_rate > 0:
        autoencoder.add(Dropout(dropout_rate, name='DO_1'))
    if activation is None:
        autoencoder.add(PReLU(name='PR_1'))

if n_l_2 > 0:
    autoencoder.add(Dense(n_l_2, activation=activation, name='E_2'))
    if dropout_rate > 0:
        autoencoder.add(Dropout(dropout_rate, name='DO_2'))
    if activation is None:
        autoencoder.add(PReLU(name='PR_2'))

if n_l_3 > 0:
    autoencoder.add(Dense(n_l_3, activation=activation, name='E_3'))
    if dropout_rate > 0:
        autoencoder.add(Dropout(dropout_rate, name='DO_3'))
    if activation is None:
        autoencoder.add(PReLU(name='PR_3'))

if n_l_4 > 0:
    autoencoder.add(Dense(n_l_4, activation=activation, name='E_4'))
    if dropout_rate > 0:
        autoencoder.add(Dropout(dropout_rate, name='DO_4'))
    if activation is None:
        autoencoder.add(PReLU(name='PR_4'))

if n_l_5 > 0:
    autoencoder.add(Dense(n_l_5, activation=activation, name='E_5'))
    if dropout_rate > 0:
        autoencoder.add(Dropout(dropout_rate, name='DO_5'))
    if activation is None:
        autoencoder.add(PReLU(name='PR_5'))

if n_l_6 > 0:
    autoencoder.add(Dense(n_l_6, activation=activation, name='E_6'))
    if dropout_rate > 0:
        autoencoder.add(Dropout(dropout_rate, name='DO_6'))
    if activation is None:
        autoencoder.add(PReLU(name='PR_6'))

autoencoder.add(Dense(n_l_e, activation=activation, name=decoded_layer_name))
if activation is None:
        autoencoder.add(PReLU(name='PR_7'))

if n_l_6 > 0:
    autoencoder.add(Dense(n_l_6, activation=activation, name='D_1'))
    if dropout_rate > 0:
        autoencoder.add(Dropout(dropout_rate, name='DO_8'))
    if activation is None:
            autoencoder.add(PReLU(name='PR_8'))

if n_l_5 > 0:
    autoencoder.add(Dense(n_l_5, activation=activation, name='D_2'))
    if dropout_rate > 0:
        autoencoder.add(Dropout(dropout_rate, name='DO_9'))
    if activation is None:
        autoencoder.add(PReLU(name='PR_9'))

if n_l_4 > 0:
    autoencoder.add(Dense(n_l_4, activation=activation, name='D_3'))
    if dropout_rate > 0:
        autoencoder.add(Dropout(dropout_rate, name='DO_10'))
    if activation is None:
        autoencoder.add(PReLU(name='PR_10'))

if n_l_3 > 0:
    autoencoder.add(Dense(n_l_3, activation=activation, name='D_4'))
    if dropout_rate > 0:
        autoencoder.add(Dropout(dropout_rate, name='DO_11'))
    if activation is None:
        autoencoder.add(PReLU(name='PR_11'))

if n_l_2 > 0:
    autoencoder.add(Dense(n_l_2, activation=activation, name='D_5'))
    if dropout_rate > 0:
        autoencoder.add(Dropout(dropout_rate, name='DO_12'))
    if activation is None:
        autoencoder.add(PReLU(name='PR_12'))

if n_l_1 > 0:
    autoencoder.add(Dense(n_l_1, activation=activation, name='D_6'))
    if dropout_rate > 0:
        autoencoder.add(Dropout(dropout_rate, name='DO_13'))
    if activation is None:
        autoencoder.add(PReLU(name='PR_13'))

autoencoder.add(Dense(n_wvl, activation='linear', name='recreated'))
autoencoder.summary()

# Visualize network architecture and save the visualization as a file
#plot_model(autoencoder, show_layer_names=True, show_shapes=True, to_file='ann_network_structure_a.pdf')
#plot_model(autoencoder, show_layer_names=True, show_shapes=True, to_file='ann_network_structure_a.png', dpi=300)

# model file handling
out_model_file = 'model_weights.h5'

if os.path.isfile(out_model_file):
    autoencoder.load_weights(out_model_file, by_name=True)


2022-04-11 18:48:56.418406: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-11 18:48:56.418451: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-11 18:48:56.418495: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (nut): /proc/driver/nvidia/version does not exist
2022-04-11 18:48:56.418829: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 E_1 (Dense)                 (None, 3344)              14914240  
                                                                 
 PR_1 (PReLU)                (None, 3344)              3344      
                                                                 
 E_2 (Dense)                 (None, 2229)              7456005   
                                                                 
 PR_2 (PReLU)                (None, 2229)              2229      
                                                                 
 E_5 (Dense)                 (None, 1114)              2484220   
                                                                 
 PR_5 (PReLU)                (None, 1114)              1114      
                                                                 
 E_6 (Dense)                 (None, 445)               4

In [24]:
plot_model(autoencoder, show_layer_names=True, show_shapes=True, to_file='ann_network_structure_a.pdf')
plot_model(autoencoder, show_layer_names=True, show_shapes=True, to_file='ann_network_structure_a.png', dpi=300)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.
You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [25]:
autoencoder.compile(optimizer=optimizer, loss='mae')

NameError: name 'optimizer' is not defined